# Trackfile original

## Configuracion

In [ ]:
import kdsource.histograms as kdh
import kdsource.surfsource as kds
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from typing import Sequence, Tuple, List, Union

plt.rcParams["figure.figsize"] = (15, 2.5)


def plot_pairwise_matrix(
    surface_source,
    vars_list: List[str],
    bins: Union[int, List[int]] = 100,
    *,
    xscale: str = "linear",
    yscale: str = "log",
    zscale: str = "log",
    figsize: Tuple[int, int] | None = None,
    plot_difference: bool = False,
    Difference_to: str = None,
    cheap: bool = True,
) -> Tuple[plt.Figure, List[List[plt.Axes]]]:
    """Draw an *n×n* matrix of distributions for every pair of variables.

    The matrix contains

    * **Diagonal** – 1-D histograms of each variable
      (`plot_distribution(vars=[v], …)`).

    * **Off-diagonals** – 2-D colour-map histograms for every
      ordered pair *(vᵢ, vⱼ)*
      (`plot_distribution(vars=[vᵢ, vⱼ], …, zscale='log')`).

    Parameters
    ----------
    surface_source
        Instance of ``kdsource.SurfaceSourceFile`` that already has the
        particles loaded.
    vars_list : list[str]
        Ordered list of variables to plot, e.g.
        ``["ln(E0/E)", "x", "y", "mu", "phi"]``.
    bins : int | list[int], default **100**
        Bin count (scalar) or per-variable list passed straight to
        ``plot_distribution``.
    xscale : {'linear', 'log'}, default **'linear'**
        Scale used for every *x-axis* in the matrix.
    yscale : {'linear', 'log'}, default **'log'**
        Scale used for every *y-axis* (and 1-D histogram counts).
    zscale : {'linear', 'log'}, default **'log'**
        Colour-bar scale for all 2-D plots.
    figsize : tuple[int, int] | None
        Figure size in inches.  Defaults to *(4 × n, 4 × n)*.

    Returns
    -------
    fig, axes
        The created ``matplotlib`` figure and its *n×n* axes matrix.

    Notes
    -----
    *Internally* uses ``plt.sca(ax)`` before calling
    ``surface_source.plot_distribution(..)`` so that the plot is drawn
    in the proper cell (``plot_distribution`` always acts on the current
    axis) :contentReference[oaicite:0]{index=0}:contentReference[oaicite:1]{index=1}.

    All 2-D plots receive ``zscale=log`` as requested :contentReference[oaicite:2]{index=2}:contentReference[oaicite:3]{index=3}.
    """
    n = len(vars_list)
    if figsize is None:
        figsize = (4 * n, 4 * n)

    fig, axes = plt.subplots(n, n, figsize=figsize, squeeze=False)

    # Ensure ``bins`` is a list the same length as ``vars_list``.
    if isinstance(bins, int):
        bins = [bins] * len(vars_list)

    for i, var_i in enumerate(vars_list):
        for j, var_j in enumerate(vars_list):
            if j < i:
                if cheap:
                    continue
            ax = axes[i][j]
            plt.sca(ax)  # make this axes current

            if i == j:
                # Diagonal: 1-D histogram
                surface_source.plot_distribution(
                    vars=[var_i],
                    bins=[bins[i]],
                    scales=[xscale, yscale],
                    plot_difference=plot_difference,
                    Difference_to=Difference_to,
                )
            else:
                # Off-diagonal: 2-D histogram
                surface_source.plot_distribution(
                    vars=[var_j, var_i],  # x = var_j, y = var_i
                    bins=[bins[j], bins[i]],
                    scales=[xscale, xscale],
                    zscale=zscale,
                    plot_difference=plot_difference,
                    Difference_to=Difference_to,
                )

            # Cosmetic tweaks: remove redundant tick labels
            if i < n - 1:
                ax.set_xlabel("")
            if j > 0:
                ax.set_ylabel("")
            if i != j:
                ax.set_title("")  # avoid clutter

    plt.tight_layout()
    return fig, axes


hs = kdh.HistogramSource(
    trackfile="/home/lucas/Documents/Proyecto_Integrador/PI/resampleo_trackfiles/trackfile2/trackfile_0.h5",
    particle_type="neutron",
    z0=0,
    Nparticles=1e10,
    # surface_geometry='circular',
    # R=5,
    domain={"w": [0, 2]},
)

print(hs.surface_source._df2["ln(E0/E)"].max())

hs.configure_binning(
    variable_order=["ln(E0/E)", "x", "y", "mu", "phi"],
    # micro_bins=[25]*5,
    micro_bins=[35, 10, 8, 4, 10],
    macro_bins=[9, 7, 5, 3],
    micro_initial_bins=[1] * 5,
    # micro_initial_bins=[None,1,1,None,None],
    macro_initial_bins=[1] * 5,
    # macro_initial_bins=[1,2,2,None,None],
    micro_binning="adaptive",
    macro_binning="adaptive",
)

hs.build_tree()

hs.write_xml(output="source.xml")

hs.generate_mcpl(
    # n_particles=5e6,
    n_particles=1e7,
    write_path="trackfile1.mcpl",
    overwrite=True,
)

## Ploteos de la distribucion original

### Ploteos 1D

In [ ]:
bins = 250
%matplotlib widget
plt.figure()
hs.surface_source.plot_distribution(
    vars=["ln(E0/E)"],
    bins=[bins],
    scales=["linear", "log"],
)
plt.grid()
plt.show()

plt.figure()
hs.surface_source.plot_distribution(
    vars=["x"],
    bins=[bins],
    scales=["linear", "log"],
)
plt.grid()
plt.show()

plt.figure()
hs.surface_source.plot_distribution(
    vars=["y"],
    bins=[bins],
    scales=["linear", "log"],
)
plt.grid()
plt.show()

plt.figure()
hs.surface_source.plot_distribution(
    vars=["mu"],
    bins=[bins],
    scales=["linear", "log"],
)
plt.grid()
plt.show()

plt.figure()
hs.surface_source.plot_distribution(
    vars=["phi"],
    bins=[bins],
    scales=["linear", "log"],
)
plt.grid()
plt.show()

### Ploteos 2D

In [ ]:
# %matplotlib widget
plt.figure()
hs.surface_source.plot_distribution(
    vars=["x", "ln(E0/E)"],
    bins=[bins, bins],
    scales=["linear", "linear"],
    zscale="log",
)
plt.show()

plt.figure()
hs.surface_source.plot_distribution(
    vars=["x", "mu"],
    bins=[bins, bins],
    scales=["linear", "linear"],
    zscale="log",
)
plt.show()

plt.figure()
hs.surface_source.plot_distribution(
    vars=["ln(E0/E)", "mu"],
    bins=[bins, bins],
    scales=["linear", "linear"],
    zscale="log",
)
plt.show()

### Ploteo nxn

In [ ]:
# %matplotlib widget
vars_to_plot = ["ln(E0/E)", "x", "y", "mu", "phi"]
fig, ax_matrix = plot_pairwise_matrix(
    hs.surface_source, vars_to_plot, bins=30,figsize=(18, 8)
)
plt.show()
plt.savefig("original_matrix.png", dpi=600, bbox_inches="tight")



# Trackfile resampleado

## Ploteo de la distribucion sampleada

### Ploteo 1D

In [ ]:
trackfile_resampled = kds.SurfaceSourceFile(
    filepath="trackfile1.mcpl.gz",
)
trackfile_resampled._Np = hs.surface_source._Np * trackfile_resampled._df2["wgt"].sum() / hs.surface_source._df2["wgt"].sum()

bins = 250
%matplotlib widget
plt.figure()
trackfile_resampled.plot_distribution(
    vars=["ln(E0/E)"],
    bins=[bins],
    scales=["linear", "log"],
)
plt.grid()
plt.show()

plt.figure()
trackfile_resampled.plot_distribution(
    vars=["x"],
    bins=[bins],
    scales=["linear", "log"],
)
plt.grid()
plt.show()

plt.figure()
trackfile_resampled.plot_distribution(
    vars=["y"],
    bins=[bins],
    scales=["linear", "log"],
)
plt.grid()
plt.show()

### Ploteo nxn

In [ ]:
fig, ax_matrix = plot_pairwise_matrix(
    trackfile_resampled, vars_to_plot, bins=30,figsize=(18, 8)
)
plt.show()
plt.savefig("resampled_matrix.png", dpi=600, bbox_inches="tight")

## Comparacion entre files

### Ploteos 1D

In [ ]:
bins = int(300)
# %matplotlib widget


def plot_comparision_1D(
    vars,
    trackfile_original=None,
    trackfile_resampled=None,
    save=False,
    right_xlim=None,
    bins=500,
    fig_name=None,
    height_ratios=[2.5, 1],
    hspace=0.3,
    figsize=(12, 6),
):

    from matplotlib.gridspec import GridSpec

    # Crear figura y gridspec con altura relativa 2.5 para el primer plot y 1 para el segundo
    fig = plt.figure(figsize=figsize)
    gs = GridSpec(2, 1, height_ratios=height_ratios, hspace=hspace)

    # Plot 1: distribuciones
    ax1 = fig.add_subplot(gs[0])
    trackfile_original.plot_distribution(
        vars=[vars],
        bins=[bins],
        scales=["linear", "log"],
        label="Trackfile original",
    )
    trackfile_resampled.plot_distribution(
        vars=[vars],
        bins=[bins],
        scales=["linear", "log"],
        label="Trackfile resampleado",
    )
    ax1.grid()
    ax1.set_ylabel("Frecuencia")
    ax1.legend()
    if right_xlim is not None:
        ax1.set_xlim(right=right_xlim)

    # Plot 2: diferencia relativa
    ax2 = fig.add_subplot(gs[1])
    trackfile_original.plot_distribution(
        vars=[vars],
        bins=[bins],
        scales=["linear", "linear"],
        plot_difference=True,
        Difference_to=trackfile_resampled,
    )
    ax2.grid()
    ax2.set_ylim(0, 100)

    if right_xlim is not None:
        ax2.set_xlim(right=right_xlim)

    plt.show()
    if save:
        plt.savefig(fig_name, dpi=600, bbox_inches="tight")

    # plt.close(fig)




plot_comparision_1D(
    "ln(E0/E)",
    trackfile_original=hs.surface_source,
    trackfile_resampled=trackfile_resampled,
    bins=bins,
    # right_xlim=4,
)
plot_comparision_1D(
    "mu",
    trackfile_original=hs.surface_source,
    trackfile_resampled=trackfile_resampled,
    # bins=1000,
    bins=bins,
    # right_xlim=4,
)
plot_comparision_1D(
    "x",
    trackfile_original=hs.surface_source,
    trackfile_resampled=trackfile_resampled,
    bins=bins,
    # bins=1000,
    # right_xlim=4,
)
plot_comparision_1D(
    "y",
    trackfile_original=hs.surface_source,
    trackfile_resampled=trackfile_resampled,
    bins=bins,
    # right_xlim=4,
)



### Ploteos 2D

In [ ]:
# %matplotlib widget
# plt.figure()
# hs.surface_source.plot_distribution(
#     vars=["x", "y"],
#     bins=[bins, bins],
#     scales=["linear", "linear"],
#     zscale="log",
# )
# plt.show()

# plt.figure()
# trackfile_resampled.plot_distribution(
#     vars=["x", "y"],
#     bins=[bins, bins],
#     scales=["linear", "linear"],
#     zscale="log",
# )
# plt.show()

def plot_comparation_2d(
    source,
    resampled,
    vars: List[str],
    bins: int,
    scales: List[str] = ["linear", "linear"],
):
    
    plt.figure()
    source.plot_distribution(
        vars=vars,
        bins=[bins, bins],
        scales=scales,
        zscale="log",
        plot_difference=True,
        Difference_to=resampled,
    )
    plt.show()

plot_comparation_2d(
    hs.surface_source,
    trackfile_resampled,
    vars=["x", "y"],
    bins=bins,
)

plot_comparation_2d(
    hs.surface_source,
    trackfile_resampled,
    vars=["x", "ln(E0/E)"],
    scales=["linear", "log"],
    bins=bins,
)
plot_comparation_2d(
    hs.surface_source,
    trackfile_resampled,
    vars=["ln(E0/E)", "y"],
    scales=["log", "linear"],
    bins=bins,
)
plot_comparation_2d(
    hs.surface_source,
    trackfile_resampled,
    scales=["log", "linear"],
    vars=["ln(E0/E)", "mu"],
    bins=bins,
)

### Matriz

In [9]:
fig, ax_matrix = plot_pairwise_matrix(
    hs.surface_source,
    vars_to_plot,
    bins=30,
    figsize=(18, 8),
    plot_difference=True,
    Difference_to=trackfile_resampled,
)
plt.show()
plt.savefig("comparision_matrix.png", dpi=600, bbox_inches="tight")

KeyboardInterrupt: 

### Divergencia KL

In [ ]:
import numpy as np
from itertools import combinations
from typing import Iterable, Mapping

# ----------------------------------------------------------------------------- 
# Helper genérico: KL sobre vectores de conteos (pseudoconteo incluido)
# -----------------------------------------------------------------------------
def _kl_counts(counts_p, counts_q, smooth: float = 0.5) -> float:
    p = counts_p.astype(float) + smooth
    q = counts_q.astype(float) + smooth
    p /= p.sum()
    q /= q.sum()
    return float(np.sum(p * np.log(p / q)))


# ----------------------------------------------------------------------------- 
# Función principal
# -----------------------------------------------------------------------------
def report_kl_divergence(
    ss_original,
    ss_resampled,
    variables: Iterable[str],
    *,
    n_bins: int = 1_000,
    smooth: float = 0.5,
    quiet: bool = False,
) -> Mapping[str, Mapping]:
    """KL 1-D y 2-D (todas las parejas) entre dos SurfaceSourceFile.

    Returns
    -------
    result : dict
        {
          "kl_1d":   {var: kl, ...},
          "kl_2d":   {"var1,var2": kl, ...},
          "total_1d": float,
          "total_2d": float,
          "total_all": float
        }
    """
    df_p = ss_original.get_pandas_dataframe()
    df_q = ss_resampled.get_pandas_dataframe()

    vars_list = list(variables)
    kl_1d, kl_2d = {}, {}
    total_1d = total_2d = 0.0

    # ---------- 1-D ----------------------------------------------------------
    for var in vars_list:
        if var not in df_p.columns or var not in df_q.columns:
            raise KeyError(f"Variable '{var}' no existe en ambos DataFrames")

        vmin = min(df_p[var].min(), df_q[var].min())
        vmax = max(df_p[var].max(), df_q[var].max())
        bins = np.linspace(vmin, vmax, n_bins + 1)

        cnt_p, _ = np.histogram(df_p[var], bins=bins)
        cnt_q, _ = np.histogram(df_q[var], bins=bins)

        kl = _kl_counts(cnt_p, cnt_q, smooth)
        kl_1d[var] = kl
        total_1d += kl

        if not quiet:
            print(f"KL 1D[{var}] = {kl:.4e} nats")

    # ---------- 2-D (todas las parejas) -------------------------------------
    for var_x, var_y in combinations(vars_list, 2):
        # rangos independientes por eje
        vx_min = min(df_p[var_x].min(), df_q[var_x].min())
        vx_max = max(df_p[var_x].max(), df_q[var_x].max())
        vy_min = min(df_p[var_y].min(), df_q[var_y].min())
        vy_max = max(df_p[var_y].max(), df_q[var_y].max())

        x_bins = np.linspace(vx_min, vx_max, n_bins + 1)
        y_bins = np.linspace(vy_min, vy_max, n_bins + 1)

        H_p, _, _ = np.histogram2d(df_p[var_x], df_p[var_y],
                                   bins=[x_bins, y_bins])
        H_q, _, _ = np.histogram2d(df_q[var_x], df_q[var_y],
                                   bins=[x_bins, y_bins])

        kl = _kl_counts(H_p.ravel(), H_q.ravel(), smooth)
        key = f"{var_x},{var_y}"
        kl_2d[key] = kl
        total_2d += kl

        if not quiet:
            print(f"KL 2D[{key}] = {kl:.4e} nats")

    total_all = total_1d + total_2d
    if not quiet:
        print("-" * 50)
        print(f"Σ KL 1-D = {total_1d:.4e}  |  Σ KL 2-D = {total_2d:.4e}")
        print(f"KL total  = {total_all:.4e} nats (1D + 2D)")

    return {
        "kl_1d": kl_1d,
        "kl_2d": kl_2d,
        "total_1d": total_1d,
        "total_2d": total_2d,
        "total_all": total_all,
    }


In [ ]:
# ss_orig y ss_resp son tus SurfaceSourceFile previamente cargados
kl_results = report_kl_divergence(
    hs.surface_source, trackfile_resampled,
    variables=["ln(E0/E)","x", "y", "mu", "phi"],
    n_bins=1000,
    smooth=1e-9,
)
